In [2]:
! pip install gensim

In [123]:
import pandas as pd
import gensim
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import math

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anyachandorkar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:
lol = pd.read_csv("critic_reviews.csv")
lol.get("review").iloc[0]
lol_sum = pd.DataFrame({'movie': (lol['movie_name']).unique()})
for i in lol_sum['movie']:
    tf_idf_df = pd.DataFrame({'DOCUMENT':lol.loc[lol['movie_name'] == i]['review']})
lol

,Unnamed: 0,index,movie_name,critics and company,date,review,score,decision
0,0,0,chicken_run,"['Dejan Jotanovic', 'Bitch Media']","March 15, 2021","Chicken Run, if anything, offers an alternativ...",[],fresh
1,1,1,chicken_run,"['Rebecca Hawkes', 'Daily Telegraph (UK)']","July 20, 2015","Its plump chickens maintain that strange, hapt...",Original Score: 5/5,fresh
2,2,2,chicken_run,"['Anthony Lane', 'New Yorker']","July 20, 2015","Even without Wallace and Gromit, most of the t...",[],fresh
3,3,3,chicken_run,"['Debra Jo Immergut', 'Wall Street Journal']","July 20, 2015","It coaxes you to laugh and cry, flatters your ...",[],fresh
4,4,4,chicken_run,"['Peter Bradshaw', 'Guardian']","August 24, 2014",Chicken Run is a charmingly conceived entertai...,[],fresh
...,...,...,...,...,...,...,...,...
41252,41252,15,mank,"['David Sims', 'The Atlantic']","December 4, 2020","Mank is a tale of triumph, the kind that's bee...",[],fresh
41253,41253,16,mank,"['A.O. Scott', 'New York Times']","December 3, 2020","A worthy, eminently watchable entry in the ann...",[],fresh
41254,41254,17,mank,"['James Berardinelli', 'ReelViews']","December 3, 2020",Although Fincher's attention to detail results...,Original Score: 3/4,fresh
41255,41255,18,mank,"['Nell Minow', 'Movie Mom']","December 3, 2020",It is pure pleasure to see a film that respect...,Original Score: A-,fresh


In [55]:
tf_idf_df

,DOCUMENT
41237,"As an example of the screenwriting art, this f..."
41238,"Mank is a love letter and a poison pen, though..."
41239,Not unlike watching Oliver Stone's JFK. It's s...
41240,"More than a magnificent technical achievement,..."
41241,"I loved looking at it, hearing it, and the act..."
41242,Fincher's latest is a disappointing slice of H...
41243,It won't be for everyone. But in the canon of ...
41244,"In his gleeful curmudgeon of a film, Fincher s..."
41245,This is masterful filmmaking about Old Hollywo...
41246,"Dor all its skill and scrupulousness, I found ..."


In [4]:
df = pd.read_csv("audience_reviews.csv")
df

,Movies,Reviews,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,rottentomatoes.com/m/chicken_run,['While Chicken Run has a great animation styl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rottentomatoes.com/m/the_taste_of_others,['A nice comedy that has pokes some friendly b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rottentomatoes.com/m/yi_yi,['A city symphony in minor key with a city-siz...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rottentomatoes.com/m/memento,"['Not a masterpiece but really good', ""Memento...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rottentomatoes.com/m/the_life_and_times_of_han...,"[""Dedicated to the millions of Detroit Tigers ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,rottentomatoes.com/m/borat_subsequent_moviefilm,"[""MINOR SPOILER ALERT\n\nBorat Subsequent Movi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2097,rottentomatoes.com/m/emma_2020,['What would be of cinema without Jane Austen?...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2098,rottentomatoes.com/m/yes_god_yes,"['That such movies are still being made, there...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2099,rottentomatoes.com/m/mank,"[""I didn't like the way it felt but the story ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def convert_movie_name(movie):
    return movie.split("/")[-1]

In [6]:
# condense df and get only movie names instead of link
df['Movies'] = df['Movies'].map(convert_movie_name)
df = df[['Movies', 'Reviews']]
df

,Movies,Reviews
0,chicken_run,['While Chicken Run has a great animation styl...
1,the_taste_of_others,['A nice comedy that has pokes some friendly b...
2,yi_yi,['A city symphony in minor key with a city-siz...
3,memento,"['Not a masterpiece but really good', ""Memento..."
4,the_life_and_times_of_hank_greenberg,"[""Dedicated to the millions of Detroit Tigers ..."
...,...,...
2096,borat_subsequent_moviefilm,"[""MINOR SPOILER ALERT\n\nBorat Subsequent Movi..."
2097,emma_2020,['What would be of cinema without Jane Austen?...
2098,yes_god_yes,"['That such movies are still being made, there..."
2099,mank,"[""I didn't like the way it felt but the story ..."


In [48]:
df.get("Reviews").iloc[0]

'[\'While Chicken Run great animation style compelling characters, plot film generic cliche me.\', "I guess I\'m minority isn\'t impressed animation. The story interesting I holocaust parallels. But didn\'t interest, short run-time.", \'On chickens symbols humanity.\\nOn fried chicken, baked chicken, pineapple chicken, lemon chicken, coconut chicken, pepper chicken, chicken soup, chicken stew, chicken salad, chicken potatoes, chicken burgers, chicken sandwich chicken parmesan.\\nYou idea.\', \'This movie way ahead it\\\'s time released. It\\\'s story "an america rooster falls love British chicken", literally story group women enslaved workers joining forces escape Nazi working camp\\nSpoilers:\\nThis movie simply amazing dynamics women, friendship, disagreement, protection, union, hopelessness... story visionary leader, willing sacrifice lead experience freedom. Her friendship Mac, woman engineer, heartwarming especially fruit hard work: plane ready.\\nRocky regular selfish overconfide

In [42]:
df.isnull()

,Movies,Reviews
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
2096,False,False
2097,False,False
2098,False,False
2099,False,False


In [87]:
def get_num_reviews(movie_idx):
    return len(df.get("Reviews").iloc[movie_idx][2:-2].replace("'", '"').split('",'))

In [113]:
stop_words = stopwords.words('english')
more_stop_words = ['.', ":", ")", "(", ",", "&", "i"]
stop_words.extend(more_stop_words)
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [114]:
def create_freq_matrix(movie_idx):
    tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    movie_revs=df.get("Reviews").iloc[movie_idx][2:-2].replace("'", '"').split('",')
    freq_matrix = {}
    for review in movie_revs:
        freq_table = {}
        words = tokenizer.tokenize(review)
        for word in words:
            word = word.lower()
            if word not in stop_words:
                if word in freq_table:
                    freq_table[word] += 1
                else:
                    freq_table[word] = 1
        freq_matrix[review] = freq_table
    return freq_matrix

In [115]:
create_freq_matrix(0)

{'While Chicken Run great animation style compelling characters, plot film generic cliche me.': {'chicken': 1,
  'run': 1,
  'great': 1,
  'animation': 1,
  'style': 1,
  'compelling': 1,
  'characters': 1,
  'plot': 1,
  'film': 1,
  'generic': 1,
  'cliche': 1},
 ' "I guess I"m minority isn"t impressed animation. The story interesting I holocaust parallels. But didn"t interest, short run-time.': {'"i': 1,
  'guess': 1,
  '"m': 1,
  'minority': 1,
  '"t': 2,
  'impressed': 1,
  'animation': 1,
  'story': 1,
  'interesting': 1,
  'holocaust': 1,
  'parallels': 1,
  'interest': 1,
  'short': 1,
  'run': 1,
  '-time.': 1},
 ' "On chickens symbols humanity.\\nOn fried chicken, baked chicken, pineapple chicken, lemon chicken, coconut chicken, pepper chicken, chicken soup, chicken stew, chicken salad, chicken potatoes, chicken burgers, chicken sandwich chicken parmesan.\\nYou idea.': {'"on': 1,
  'chickens': 1,
  'symbols': 1,
  'humanity': 1,
  '.\\non': 1,
  'fried': 1,
  'chicken': 13,
 

In [116]:
def create_tf_matrix(movie_idx):
    tf_matrix = {}

    for sent, f_table in create_freq_matrix(movie_idx).items():
        tf_table = {}

        num_words = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / num_words

        tf_matrix[sent] = tf_table

    return tf_matrix

In [117]:
create_tf_matrix(0)

{'While Chicken Run great animation style compelling characters, plot film generic cliche me.': {'chicken': 0.09090909090909091,
  'run': 0.09090909090909091,
  'great': 0.09090909090909091,
  'animation': 0.09090909090909091,
  'style': 0.09090909090909091,
  'compelling': 0.09090909090909091,
  'characters': 0.09090909090909091,
  'plot': 0.09090909090909091,
  'film': 0.09090909090909091,
  'generic': 0.09090909090909091,
  'cliche': 0.09090909090909091},
 ' "I guess I"m minority isn"t impressed animation. The story interesting I holocaust parallels. But didn"t interest, short run-time.': {'"i': 0.06666666666666667,
  'guess': 0.06666666666666667,
  '"m': 0.06666666666666667,
  'minority': 0.06666666666666667,
  '"t': 0.13333333333333333,
  'impressed': 0.06666666666666667,
  'animation': 0.06666666666666667,
  'story': 0.06666666666666667,
  'interesting': 0.06666666666666667,
  'holocaust': 0.06666666666666667,
  'parallels': 0.06666666666666667,
  'interest': 0.06666666666666667,

In [126]:
def count_doc_per_word(movie_idx):
    word_per_doc_table = {}
    
    for sent, f_table in create_freq_matrix(movie_idx).items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [120]:
count_doc_per_word(0)

{'chicken': 4,
 'run': 3,
 'great': 1,
 'animation': 3,
 'style': 2,
 'compelling': 1,
 'characters': 1,
 'plot': 1,
 'film': 2,
 'generic': 1,
 'cliche': 1,
 '"i': 2,
 'guess': 1,
 '"m': 2,
 'minority': 1,
 '"t': 1,
 'impressed': 1,
 'story': 4,
 'interesting': 1,
 'holocaust': 1,
 'parallels': 1,
 'interest': 1,
 'short': 1,
 '-time.': 1,
 '"on': 1,
 'chickens': 3,
 'symbols': 1,
 'humanity': 1,
 '.\\non': 1,
 'fried': 1,
 'baked': 1,
 'pineapple': 1,
 'lemon': 1,
 'coconut': 1,
 'pepper': 1,
 'soup': 1,
 'stew': 1,
 'salad': 1,
 'potatoes': 1,
 'burgers': 1,
 'sandwich': 1,
 'parmesan': 1,
 '.\\nyou': 1,
 'idea': 1,
 '"this': 1,
 'movie': 4,
 'way': 1,
 'ahead': 1,
 '\\"s': 1,
 'time': 1,
 'released': 1,
 '"an': 1,
 'america': 1,
 'rooster': 1,
 'falls': 1,
 'love': 1,
 'british': 1,
 'literally': 1,
 'group': 1,
 'women': 1,
 'enslaved': 1,
 'workers': 1,
 'joining': 1,
 'forces': 1,
 'escape': 2,
 'nazi': 1,
 'working': 1,
 'camp': 1,
 '\\nspoilers:\\nthis': 1,
 'simply': 1,
 'ama

In [129]:
def create_idf_matrix(movie_idx):
    idf_matrix = {}

    for sent, f_table in create_freq_matrix(movie_idx).items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(get_num_reviews(movie_idx) / float(count_doc_per_word(movie_idx)[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [130]:
create_idf_matrix(0)

{'While Chicken Run great animation style compelling characters, plot film generic cliche me.': {'chicken': 0.43933269383026263,
  'run': 0.5642714304385625,
  'great': 1.0413926851582251,
  'animation': 0.5642714304385625,
  'style': 0.7403626894942439,
  'compelling': 1.0413926851582251,
  'characters': 1.0413926851582251,
  'plot': 1.0413926851582251,
  'film': 0.7403626894942439,
  'generic': 1.0413926851582251,
  'cliche': 1.0413926851582251},
 ' "I guess I"m minority isn"t impressed animation. The story interesting I holocaust parallels. But didn"t interest, short run-time.': {'"i': 0.7403626894942439,
  'guess': 1.0413926851582251,
  '"m': 0.7403626894942439,
  'minority': 1.0413926851582251,
  '"t': 1.0413926851582251,
  'impressed': 1.0413926851582251,
  'animation': 0.5642714304385625,
  'story': 0.43933269383026263,
  'interesting': 1.0413926851582251,
  'holocaust': 1.0413926851582251,
  'parallels': 1.0413926851582251,
  'interest': 1.0413926851582251,
  'short': 1.0413926

In [131]:
def create_tfidf_matrix(movie_idx):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(create_tf_matrix(movie_idx).items(), create_idf_matrix(movie_idx).items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()): 
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [133]:
create_tfidf_matrix(0)

{'While Chicken Run great animation style compelling characters, plot film generic cliche me.': {'chicken': 0.03993933580275115,
  'run': 0.05129740276714205,
  'great': 0.09467206228711138,
  'animation': 0.05129740276714205,
  'style': 0.06730569904493126,
  'compelling': 0.09467206228711138,
  'characters': 0.09467206228711138,
  'plot': 0.09467206228711138,
  'film': 0.06730569904493126,
  'generic': 0.09467206228711138,
  'cliche': 0.09467206228711138},
 ' "I guess I"m minority isn"t impressed animation. The story interesting I holocaust parallels. But didn"t interest, short run-time.': {'"i': 0.049357512632949595,
  'guess': 0.06942617901054834,
  '"m': 0.049357512632949595,
  'minority': 0.06942617901054834,
  '"t': 0.1388523580210967,
  'impressed': 0.06942617901054834,
  'animation': 0.03761809536257084,
  'story': 0.02928884625535084,
  'interesting': 0.06942617901054834,
  'holocaust': 0.06942617901054834,
  'parallels': 0.06942617901054834,
  'interest': 0.06942617901054834

In [151]:
def score_sentences(movie_idx):
    sentenceValue = {}

    for sent, f_table in create_tfidf_matrix(movie_idx).items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

In [152]:
score_sentences(0)

{'While Chicken Run great animation style compelling characters, plot film generic cliche me.': 0.07683435574086962,
 ' "I guess I"m minority isn"t impressed animation. The story interesting I holocaust parallels. But didn"t interest, short run-time.': 0.0644618687574949,
 ' "On chickens symbols humanity.\\nOn fried chicken, baked chicken, pineapple chicken, lemon chicken, coconut chicken, pepper chicken, chicken soup, chicken stew, chicken salad, chicken potatoes, chicken burgers, chicken sandwich chicken parmesan.\\nYou idea.': 0.05909763598239967,
 ' "This movie way ahead it\\"s time released. It\\"s story "an america rooster falls love British chicken': 0.06602712439354542,
 ' literally story group women enslaved workers joining forces escape Nazi working camp\\nSpoilers:\\nThis movie simply amazing dynamics women, friendship, disagreement, protection, union, hopelessness... story visionary leader, willing sacrifice lead experience freedom. Her friendship Mac, woman engineer, heart

In [136]:
def find_avg_score(movie_idx):
    sum_values = 0
    sentence_val = score_sentences(movie_idx)
    for entry in sentence_val:
        sum_values += sentence_val[entry]

    average = (sum_values / len(sentence_val))

    return average

In [155]:
find_avg_score(0)

0.129932758289126

In [184]:
def generate_summary(movie_idx):
    sentence_count = 0
    summary = ''
    movie_revs=df.get("Reviews").iloc[movie_idx][2:-2].replace("'", '"').split('",')
    for review in movie_revs:
        if review in score_sentences(movie_idx) and score_sentences(movie_idx)[review] >= (1.3*find_avg_score(movie_idx)):
            summary += " " + review
            sentence_count += 1
    return summary

In [185]:
score_sentences(0)

{'While Chicken Run great animation style compelling characters, plot film generic cliche me.': 0.07683435574086962,
 ' "I guess I"m minority isn"t impressed animation. The story interesting I holocaust parallels. But didn"t interest, short run-time.': 0.0644618687574949,
 ' "On chickens symbols humanity.\\nOn fried chicken, baked chicken, pineapple chicken, lemon chicken, coconut chicken, pepper chicken, chicken soup, chicken stew, chicken salad, chicken potatoes, chicken burgers, chicken sandwich chicken parmesan.\\nYou idea.': 0.05909763598239967,
 ' "This movie way ahead it\\"s time released. It\\"s story "an america rooster falls love British chicken': 0.06602712439354542,
 ' literally story group women enslaved workers joining forces escape Nazi working camp\\nSpoilers:\\nThis movie simply amazing dynamics women, friendship, disagreement, protection, union, hopelessness... story visionary leader, willing sacrifice lead experience freedom. Her friendship Mac, woman engineer, heart

In [186]:
generate_summary(1)

' A nice comedy pokes friendly barbs social class conventions, utterly charming.  "Very overrated, extremely overrated good drama comedy. Good directing acting drags bit.  "Unexciting character study. Their quirks provide amusing moments drags bit.'

In [187]:
def create_movie_summaries():
    summaries = {}
    for i in range(len(df.index)):
        summaries[df.get("Movies").iloc[i]] = generate_summary(i)
    return summaries

In [ ]:
create_movie_summaries()

In [157]:
df.get("Reviews").iloc[0][2:-2].replace("'", '"').split('",')

['While Chicken Run great animation style compelling characters, plot film generic cliche me.',
 ' "I guess I"m minority isn"t impressed animation. The story interesting I holocaust parallels. But didn"t interest, short run-time.',
 ' "On chickens symbols humanity.\\nOn fried chicken, baked chicken, pineapple chicken, lemon chicken, coconut chicken, pepper chicken, chicken soup, chicken stew, chicken salad, chicken potatoes, chicken burgers, chicken sandwich chicken parmesan.\\nYou idea.',
 ' "This movie way ahead it\\"s time released. It\\"s story "an america rooster falls love British chicken',
 ' literally story group women enslaved workers joining forces escape Nazi working camp\\nSpoilers:\\nThis movie simply amazing dynamics women, friendship, disagreement, protection, union, hopelessness... story visionary leader, willing sacrifice lead experience freedom. Her friendship Mac, woman engineer, heartwarming especially fruit hard work: plane ready.\\nRocky regular selfish overconfid

In [183]:
df.get("Movies").iloc[0]

'chicken_run'